In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
from src import supabase_client
from src.update_pipeline import update_training_week
from src.types.update_pipeline import ExeType
from src.utils import get_last_sunday
from src import auth_manager
from src.activities import get_weekly_summaries, get_daily_activity
from src.auth_manager import get_strava_client
from src.llm import get_completion_json
from src.constants import COACH_ROLE
from src.utils import datetime_now_est

from enum import StrEnum
from freezegun import freeze_time
import os
import numpy as np
from pydantic import BaseModel, Field
from datetime import timedelta
import matplotlib.pyplot as plt

plt.style.use("seaborn-v0_8")

# Update Jamie's Training Plan

In [6]:
user = supabase_client.get_user(os.environ["JAMIES_ATHLETE_ID"])
assert user.preferences.race_date is not None
assert user.preferences.race_distance is not None

@freeze_time(f"{get_last_sunday()} 18:00:00")
def frozen_update_training_week_new_week():
    return update_training_week(user, ExeType.NEW_WEEK)

response = frozen_update_training_week_new_week()

INFO:root:Sending push notification with token: b19664f8...
INFO:httpx:HTTP Request: POST https://api.push.apple.com/3/device/b19664f8a44962efa4ff49057377f565f77111310d3600e8ed26c7877ff0ca3b "HTTP/2 403 Forbidden"
ERROR:root:APNs error: 403, {'reason': 'ExpiredProviderToken'}
ERROR:root:Failed to send push notification to user 98390356: APNs rejected the request: {'reason': 'ExpiredProviderToken'}
ERROR:root:Traceback (most recent call last):
  File "/Users/jamievoynow/Desktop/code/trackflow/api/src/apn.py", line 79, in send_push_notification
    response.raise_for_status()
  File "/Users/jamievoynow/Desktop/code/trackflow/api/.venv/lib/python3.11/site-packages/httpx/_models.py", line 763, in raise_for_status
    raise HTTPStatusError(message, request=request, response=self)
httpx.HTTPStatusError: Client error '403 Forbidden' for url 'https://api.push.apple.com/3/device/b19664f8a44962efa4ff49057377f565f77111310d3600e8ed26c7877ff0ca3b'
For more information check: https://developer.mozil

{'week_of_year': 50, 'year': 2024, 'thoughts': "This week aims to continue the client's adaptation to higher mileage, maximizing their aerobic base. Given the recent history, the aim is to consolidate around the upper percentiles of recent mileage.", 'total_volume': 50, 'long_run': 10, 'athlete_id': 98390356, 'created_at': '2024-12-10T14:15:00.446181'}


ValueError: APNs rejected the request: {'reason': 'ExpiredProviderToken'}

# New Week Update Training Week with Side Effects

In [ ]:
@freeze_time(f"{get_last_sunday()} 18:00:00")
def frozen_update_training_week_new_week(user):
    return update_training_week(user, ExeType.NEW_WEEK)

for user in supabase_client.list_users(debug=True):
    auth_manager.authenticate_athlete(user.athlete_id)
    try:
        response = frozen_update_training_week_new_week(user)
        print(response)
    except Exception as e:
        print(e)

# Mid Week Update Training Week with Side Effects

In [ ]:
user = supabase_client.get_user(os.environ["JAMIES_ATHLETE_ID"])
auth_manager.authenticate_athlete(user.athlete_id)
response = update_training_week(user, ExeType.MID_WEEK)
print(response)